##  Automating Everything

In this lab, we will learn how to use **LLMOps with Prompt Flow** Solution Accelerator to deploy an end-to-end LLMOps solution using **Prompt Flow** and **Github Actions**.

### Prerequisites

- Azure OpenAI with Model deployed with name `gpt-35-turbo`.

### 1. Create Azure service principal

An Azure service principal (SP) is a special type of identity that can be used by automated tools to access Azure resources. 

We will use a service principal to grant GitHub Actions the permission to use the resources in our Azure subscription.

<span style="color:teal;">Run the following bash script</span>, updating the **spname** and **subscriptionId** variables with the values for your project to create the SP. 

In [ ]:
%%bash

# variables
spname="LLMOps-$(date +%s)"
roleName="Owner"
subscriptionId="<subscription Id>"
servicePrincipalName="Azure-ARM-${spname}"

echo "Using subscription ID $subscriptionID"
echo "Creating SP for RBAC with name $servicePrincipalName, with role $roleName and in scopes /subscriptions/$subscriptionId"
az ad sp create-for-rbac --name $servicePrincipalName --role $roleName --scopes /subscriptions/$subscriptionId --sdk-auth 
echo "Please ensure that the information created here is properly save for future use."


> The previous ```az ad create-for-rbac ...``` command will also grant the *Contributor* role to the service principal in the subscription provided. <strong>Contributor or Owner?</strong>

After running the script, you'll be presented with information related to the service principal. 

<span style="color:teal;">Save this information to a safe location</span>, you'll use it later in the GitHub configuration.

### 2. Setup runtime for Prompt flow

Prompt Flow 'flows' require runtime associated with compute instance in Azure Machine Learning workspace. 

Before we run any Prompt Flow, we need to create two things using the Service Principal: a Compute Instance and a Prompt Flow runtime. 

This ensures that Service Principal is the owner of these resources and Flows can be executed on them from Github workflows.

<span style="color:teal;">Run the following bash script</span>, updating the variables with the values for your project.

In [ ]:
%%bash

# variables
subscriptionId=<your azure subscription id>
rgname=<your resource group name>
workspace_name=<your Azure machine learning workspace name> 
userAssignedId=<enter user assigned managed identifier name>
keyvault=<your Azure machine learning workspace associate key vault name>
compute_name=<enter compute name>
location=<your Azure machine learning workspace region>
runtimeName=<enter runtime name>
sp_id=<your azure service principal or client id>
sp_password=<your service principal password>
tenant_id=<your azure tenant id>

# Create a user-assigned managed identity
echo "# Creating user-assigned managed identity"
az identity create -g $rgname -n $userAssignedId --query "id"

# Get id, principalId of user-assigned managed identity
echo "# Getting id, principalId of user-assigned managed identity"
um_details=$(az identity show -g $rgname -n $userAssignedId --query "[id, clientId, principalId]")

# Get id of user-assigned managed identity
echo "# Getting id of user-assigned managed identity"
# user_managed_id="$(echo $um_details | jq -r '.[0]')"
user_managed_id=$(echo $um_details | sed 's/[][]//g' | awk -F'"' '{print $2}')

# Get principal Id of user-assigned managed identity
echo "# Getting principal Id of user-assigned managed identity"
# principalId="$(echo $um_details | jq -r '.[2]')"
principalId=$(echo $um_details | sed 's/[][]//g' | awk -F'"' '{print $6}')

# Grant the user managed identity permission to access the workspace (AzureML Data Scientist)
echo "# Granting the user managed identity permission to access the workspace (AzureML Data Scientist)"
az role assignment create --assignee $principalId --role "AzureML Data Scientist" --scope "/subscriptions/$subscriptionId/resourcegroups/$rgname/providers/Microsoft.MachineLearningServices/workspaces/$workspace_name"

# Grant the user managed identity permission to access the workspace keyvault (get and list)
echo "# Granting the user managed identity permission to access the workspace keyvault (get and list)"
az keyvault set-policy --name $keyvault --resource-group $rgname --object-id $principalId --secret-permissions get list

# Login with Service Principal
echo "# Login with Service Principal"
az login --service-principal -u $sp_id -p $sp_password --tenant $tenant_id
az account set -s $subscriptionId

# Create compute instance and assign user managed identity to it
echo "# Creating compute instance and assign user managed identity to it"
az ml compute create --name $compute_name --size Standard_E4s_v3 --identity-type UserAssigned --type ComputeInstance --resource-group $rgname --workspace-name $workspace_name --user-assigned-identities $user_managed_id

# Get Service Principal Azure Entra token for REST API
echo "# Getting Service Principal Azure Entra token for REST API"
# access_token=$(az account get-access-token | jq -r ".accessToken")
access_token=$(az account get-access-token | grep -oP '"accessToken": "\K[^"]*')

# Construct POST url for runtime
echo "# Constructing POST url for runtime"
runtime_url_post=$(echo "https://ml.azure.com/api/$location/flow/api/subscriptions/$subscriptionId/resourceGroups/$rgname/providers/Microsoft.MachineLearningServices/workspaces/$workspace_name/FlowRuntimes/$runtimeName?asyncCall=true")

# Construct GET url for runtime
echo "# Constructing GET url for runtime"
runtime_url_get=$(echo "https://ml.azure.com/api/$location/flow/api/subscriptions/$subscriptionId/resourceGroups/$rgname/providers/Microsoft.MachineLearningServices/workspaces/$workspace_name/FlowRuntimes/$runtimeName")

# Create runtime using REST API
echo "# Creating runtime using REST API"
curl --request POST \
  --url "$runtime_url_post" \
  --header "Authorization: Bearer $access_token" \
  --header 'Content-Type: application/json' \
  --data "{
    \"runtimeType\": \"ComputeInstance\",
    \"computeInstanceName\": \"$compute_name\",
}"

# Get runtime creation status using REST API
echo "\n# Getting runtime creation status using REST API"
curl --request GET \
  --url "$runtime_url_get" \
  --header "Authorization: Bearer $access_token"

### 3. Set up Github Repo

TBD

### 4. Set up authentication with Azure and GitHub

TBD

### 5. Setup connections for Prompt flow

TBD

### 6. Cloning the repo

TBD

### 7. Update configurations for Prompt flow and GitHub Actions

TBD

### 8. Prompt Run, Evaluation and Deployment

TBD

### 9. Moving to production

TBD